## Oppgaver
1. Finn antall unike serveringssted (bruk navn og orgnummer), se [her](https://docs.pola.rs/py-polars/html/reference/dataframe/api/polars.DataFrame.unique.html)
2. Lag en ny kolonne som er true hvis kontrollen er surmunn (bruk total_karakter), false hvis ikke. Kall kolonnen "surmunn". Se: [her](https://docs.pola.rs/py-polars/html/reference/expressions/api/polars.Expr.alias.html#polars.Expr.alias) og [her](https://docs.pola.rs/py-polars/html/reference/expressions/api/polars.Expr.eq.html) 
3. Lag en ny kolonne med årstallet for kontrollen. Kall denne kolonnen for "år". 
4. Tell antall kontroller pr. serveringssted, se [her](https://docs.pola.rs/py-polars/html/reference/dataframe/api/polars.dataframe.group_by.GroupBy.agg.html#polars.dataframe.group_by.GroupBy.agg)
5. Finn serveringsstedet med flest antall kontroller.
6. Finn serveringsstedet med flest antall kontroller i et år - hint: bruk pl.col("dato").dt.year() i grupperingen. 
7. Tell antall surmunn pr. serveringssted, sorter synkende på antall surmunn. Hint: bruk sum over kolonnen surmunn i en groupby agg. Se [her](https://docs.pola.rs/py-polars/html/reference/expressions/api/polars.Expr.sum.html#polars.Expr.sum)
8. Finn antall surmunn eller strekmunn pr. serveringssted. 
9. Finn tilsynene i Tønsberg eller Horten, nyeste tilsyn først. 
10. Finn tilsyn i Tønsberg eller Horten hvor det er gitt karakteren 3 eller 4 til Renhold. Her må du joine med kravpunktene. Se oppvarmingen.  
11. Finn tilsyn i Tønsberg eller Horten hvor det er gitt surmunn som totalkarakter, og samle opp kravpunktene som har fått surmunn 3 i en liste. Se eksempelet [her](https://docs.pola.rs/py-polars/html/reference/dataframe/api/polars.dataframe.group_by.GroupBy.agg.html#polars.dataframe.group_by.GroupBy.agg)

12. I denne oppgaven skal du bruke lazy evaluering. Start med å bruke scan_csv til å lese inn csvene. Deretter skal du lage en rapport over seneste kontrollene i Tønsberg og Horten hvor det er gitt strekmunn eller surmunn. Rapporten skal inneholde karakteren som er gitt, en liste over kravpunktene som har strekmunn i en av kolonnene, en liste over kravpunktene som har surmunn i en av kolonnene. Rapporten skal være sortert nyeste kontroll først. 
13. Utvid oppgaven over ved å inkludere totalkarakteren på forrige kontroll dersom den finnes i rapporten.

In [1]:
import polars as pl

In [7]:
df_t = pl.scan_csv("tilsyn.csv", separator=";", dtypes={"postnr":str, "dato":str}).with_columns(
    pl.col("dato").str.to_date(format="%d%m%Y")
).collect()
df_t.head(3)

tilsynsobjektid,orgnummer,navn,adrlinje1,adrlinje2,postnr,poststed,tilsynid,sakref,status,dato,total_karakter,tilsynsbesoektype,tema1_no,tema1_nn,karakter1,tema2_no,tema2_nn,karakter2,tema3_no,tema3_nn,karakter3,tema4_no,tema4_nn,karakter4
str,i64,str,str,str,str,str,str,str,i64,date,i64,i64,str,str,i64,str,str,i64,str,str,i64,str,str,i64
"""Z1501071230082…",914541034,"""Bakeriet i Bru…","""VANGSVEGEN 62""",null,"""2317""","""HAMAR""","""Z1601211510285…","""2016/6663""",0,2016-01-21,1,0,"""Rutiner og led…","""Rutiner og lei…",1,"""Lokaler og uts…","""Lokaler og uts…",1,"""Mat-håndtering…","""Mat-handtering…",1,"""Merking og spo…","""Merking og spo…",0
"""Z1006120609250…",987770740,"""Aktiven skihei…","""Kvamskogen""",null,"""5600""","""NORHEIMSUND""","""Z1603101430592…","""2016/53185""",0,2016-03-10,1,0,"""Rutiner og led…","""Rutiner og lei…",0,"""Lokaler og uts…","""Lokaler og uts…",0,"""Mat-håndtering…","""Mat-handtering…",0,"""Merking og spo…","""Merking og spo…",1
"""Z1107281100559…",998066662,"""Golfkafeen""","""Belsjøveien 50…",null,"""1445""","""DRØBAK""","""Z1605111530190…","""2016/106865""",0,2016-05-11,1,0,"""Rutiner og led…","""Rutiner og lei…",0,"""Lokaler og uts…","""Lokaler og uts…",1,"""Mat-håndtering…","""Mat-handtering…",0,"""Merking og spo…","""Merking og spo…",1


In [5]:
#Finn antall unike serveringssted (bruk navn og orgnummer) 
df_t.unique(subset=["navn", "orgnummer"]).count()

tilsynsobjektid,orgnummer,navn,adrlinje1,adrlinje2,postnr,poststed,tilsynid,sakref,status,dato,total_karakter,tilsynsbesoektype,tema1_no,tema1_nn,karakter1,tema2_no,tema2_nn,karakter2,tema3_no,tema3_nn,karakter3,tema4_no,tema4_nn,karakter4
u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32
8988,8964,8988,8960,70,8986,8986,8988,8988,8988,8988,8988,8988,8988,8988,8973,8988,8988,8987,8988,8988,8984,8988,8988,8977


In [12]:
#2 Lag en ny kolonne som er true hvis kontrollen er surmunn (bruk total_karakter), false hvis ikke. Kall kolonnen "surmunn".
df_t = df_t.with_columns(
    surmunn=pl.col("total_karakter").eq(pl.lit(3))
)
df_t.head(2)

tilsynsobjektid,orgnummer,navn,adrlinje1,adrlinje2,postnr,poststed,tilsynid,sakref,status,dato,total_karakter,tilsynsbesoektype,tema1_no,tema1_nn,karakter1,tema2_no,tema2_nn,karakter2,tema3_no,tema3_nn,karakter3,tema4_no,tema4_nn,karakter4,surmunn
str,i64,str,str,str,str,str,str,str,i64,date,i64,i64,str,str,i64,str,str,i64,str,str,i64,str,str,i64,bool
"""Z1501071230082…",914541034,"""Bakeriet i Bru…","""VANGSVEGEN 62""",null,"""2317""","""HAMAR""","""Z1601211510285…","""2016/6663""",0,2016-01-21,1,0,"""Rutiner og led…","""Rutiner og lei…",1,"""Lokaler og uts…","""Lokaler og uts…",1,"""Mat-håndtering…","""Mat-handtering…",1,"""Merking og spo…","""Merking og spo…",0,false
"""Z1006120609250…",987770740,"""Aktiven skihei…","""Kvamskogen""",null,"""5600""","""NORHEIMSUND""","""Z1603101430592…","""2016/53185""",0,2016-03-10,1,0,"""Rutiner og led…","""Rutiner og lei…",0,"""Lokaler og uts…","""Lokaler og uts…",0,"""Mat-håndtering…","""Mat-handtering…",0,"""Merking og spo…","""Merking og spo…",1,false


In [15]:
#3 Lag en ny kolonne med årstallet for kontrollen. Kall denne kolonnen for "år".
df_t = df_t.with_columns(
    år=pl.col("dato").dt.year(),
)
df_t.head(2)

tilsynsobjektid,orgnummer,navn,adrlinje1,adrlinje2,postnr,poststed,tilsynid,sakref,status,dato,total_karakter,tilsynsbesoektype,tema1_no,tema1_nn,karakter1,tema2_no,tema2_nn,karakter2,tema3_no,tema3_nn,karakter3,tema4_no,tema4_nn,karakter4,surmunn,år
str,i64,str,str,str,str,str,str,str,i64,date,i64,i64,str,str,i64,str,str,i64,str,str,i64,str,str,i64,bool,i32
"""Z1501071230082…",914541034,"""Bakeriet i Bru…","""VANGSVEGEN 62""",null,"""2317""","""HAMAR""","""Z1601211510285…","""2016/6663""",0,2016-01-21,1,0,"""Rutiner og led…","""Rutiner og lei…",1,"""Lokaler og uts…","""Lokaler og uts…",1,"""Mat-håndtering…","""Mat-handtering…",1,"""Merking og spo…","""Merking og spo…",0,false,2016
"""Z1006120609250…",987770740,"""Aktiven skihei…","""Kvamskogen""",null,"""5600""","""NORHEIMSUND""","""Z1603101430592…","""2016/53185""",0,2016-03-10,1,0,"""Rutiner og led…","""Rutiner og lei…",0,"""Lokaler og uts…","""Lokaler og uts…",0,"""Mat-håndtering…","""Mat-handtering…",0,"""Merking og spo…","""Merking og spo…",1,false,2016


In [16]:
#4 Tell antall kontroller pr. serveringssted 
df_t.group_by("orgnummer", "navn").agg(
    pl.col("tilsynid").count().alias("antall_kontroller")
)

orgnummer,navn,antall_kontroller
i64,str,u32
914541034,"""Bakeriet i Bru…",10
987046848,"""Kjelstad Baker…",7
997644832,"""Café Lasse""",9
972805025,"""Kafe Edgar""",6
984934580,"""Bliss Consept …",2
974052393,"""Magnussen Kaff…",9
994811428,"""Lofoten Fisker…",7
816392012,"""Pronto Pizza A…",7
989541137,"""Park cafeen""",2


In [18]:
#5 Finn serveringsstedet med flest antall kontroller.
df_t.group_by("orgnummer", "navn").agg(
    pl.col("tilsynid").count().alias("antall_kontroller")
).sort("antall_kontroller", descending=True).head(7)

orgnummer,navn,antall_kontroller
i64,str,u32
917820902,"""FAST AND TASTY…",23
974004267,"""Nye Kro og Piz…",22
914275857,"""(Ikke gyldig, …",22
815977882,"""Kebabish Origi…",20
997605764,"""Teater Kebab""",19
972311308,"""Punjabi Masala…",19
993386642,"""Beijing Restau…",19


In [21]:
#6. Finn serveringsstedet med flest antall kontroller i et år - hint: bruk pl.col("dato").dt.year() i grupperingen.
df_t.group_by("orgnummer", "navn", "år").agg(
    pl.col("tilsynid").count().alias("antall_kontroller")
).sort("antall_kontroller", descending=True).head(10)

orgnummer,navn,år,antall_kontroller
i64,str,i32,u32
974004267,"""Nye Kro og Piz…",2018,11
815977882,"""Kebabish Origi…",2016,8
918969497,"""Taj India""",2023,8
920375944,"""St bar m Zafar…",2023,8
995243199,"""Veitvet Sportb…",2018,8
917820902,"""FAST AND TASTY…",2017,8
922531188,"""Hos Martin Øst…",2023,7
899607562,"""Dovrestua """,2017,7
919785101,"""Raudals Kro v.…",2019,7


In [22]:
# 7. Tell antall surmunn pr. serveringssted, sorter synkende på antall surmunn. Hint: bruk sum over kolonnen surmunn i en groupby agg. 
df_t.group_by("orgnummer", "navn").agg(
    pl.col("surmunn").sum().alias("antall_surmunn")
).sort("antall_surmunn", descending=True)

orgnummer,navn,antall_surmunn
i64,str,u32
995243199,"""Veitvet Sportb…",6
921187777,"""Oslo kebab & p…",6
913966244,"""Sanjays AS""",5
921642571,"""Mikado""",4
914275857,"""(Ikke gyldig, …",4
915170390,"""Høland`s Pizza…",4
922002894,"""Savanna""",4
927025183,"""Nem. Vietnames…",4
924536551,"""Valeri Kebab""",4


In [23]:
# 8 Finn antall surmunn eller strekmunn pr. serveringssted.
df_t.group_by("orgnummer", "navn").agg(
    pl.col("total_karakter").is_in([2,3]).sum().alias("antall_surmunn_eller_strekmunn")
).sort("antall_surmunn_eller_strekmunn", descending=True)

orgnummer,navn,antall_surmunn_eller_strekmunn
i64,str,u32
914275857,"""(Ikke gyldig, …",15
974004267,"""Nye Kro og Piz…",13
917820902,"""FAST AND TASTY…",12
972311308,"""Punjabi Masala…",12
995243199,"""Veitvet Sportb…",12
919785101,"""Raudals Kro v.…",11
899607562,"""Dovrestua """,11
974432234,"""Tukthuset Mat …",11
813412012,"""Baba sushi""",11


In [25]:
# 9 Finn tilsynene i Tønsberg eller Horten, nyeste tilsyn først. 
df_t.filter(
    pl.col("poststed").is_in(["TØNSBERG", "HORTEN"])
).sort(pl.col("dato"), descending=True)

tilsynsobjektid,orgnummer,navn,adrlinje1,adrlinje2,postnr,poststed,tilsynid,sakref,status,dato,total_karakter,tilsynsbesoektype,tema1_no,tema1_nn,karakter1,tema2_no,tema2_nn,karakter2,tema3_no,tema3_nn,karakter3,tema4_no,tema4_nn,karakter4,surmunn,år
str,i64,str,str,str,str,str,str,str,i64,date,i64,i64,str,str,i64,str,str,i64,str,str,i64,str,str,i64,bool,i32
"""Z1909261320111…",916942931,"""Kontoret""","""Storgaten 30""",null,"""3126""","""TØNSBERG""","""Z2401191036505…","""2024/7310""",0,2024-01-19,0,1,"""Rutiner og led…","""Rutiner og lei…",0,"""Lokaler og uts…","""Lokaler og uts…",0,"""Mat-håndtering…","""Mat-handtering…",1,"""Merking og spo…","""Merking og spo…",null,false,2024
"""Z1909261320111…",916942931,"""Kontoret""","""Storgaten 30""",null,"""3126""","""TØNSBERG""","""Z2401111324403…","""2024/7310""",0,2024-01-11,2,0,"""Rutiner og led…","""Rutiner og lei…",2,"""Lokaler og uts…","""Lokaler og uts…",2,"""Mat-håndtering…","""Mat-handtering…",2,"""Merking og spo…","""Merking og spo…",null,false,2024
"""Z2006101112483…",995084929,"""Lucky Bowl, Ho…","""Borreveien 42""",null,"""3186""","""HORTEN""","""Z2310171505090…","""2023/222122""",0,2023-10-17,1,0,"""Rutiner og led…","""Rutiner og lei…",0,"""Lokaler og uts…","""Lokaler og uts…",1,"""Mat-håndtering…","""Mat-handtering…",0,"""Merking og spo…","""Merking og spo…",0,false,2023
"""Z2309251213012…",932199327,"""Traktern AS""","""Falsens gate 1…",null,"""3187""","""HORTEN""","""Z2310121408593…","""2023/218688""",0,2023-10-12,0,0,"""Rutiner og led…","""Rutiner og lei…",0,"""Lokaler og uts…","""Lokaler og uts…",0,"""Mat-håndtering…","""Mat-handtering…",0,"""Merking og spo…","""Merking og spo…",0,false,2023
"""Z1505120919157…",971945389,"""Nye Bamboo Gar…","""Øvre Langgate …",null,"""3110""","""TØNSBERG""","""Z2310051608534…","""2023/213328""",0,2023-10-05,1,0,"""Rutiner og led…","""Rutiner og lei…",0,"""Lokaler og uts…","""Lokaler og uts…",1,"""Mat-håndtering…","""Mat-handtering…",1,"""Merking og spo…","""Merking og spo…",1,false,2023
"""Z1703031027480…",979536461,"""Cafe Eiktoppe…","""EIKVEIEN 49""",null,"""3122""","""TØNSBERG""","""Z2309281325030…","""2023/207250""",0,2023-09-28,1,0,"""Rutiner og led…","""Rutiner og lei…",0,"""Lokaler og uts…","""Lokaler og uts…",1,"""Mat-håndtering…","""Mat-handtering…",0,"""Merking og spo…","""Merking og spo…",0,false,2023
"""Z2112062350006…",987446722,"""Alfredo Rossi""","""Vektergården""","""Langgata 2""","""3188""","""HORTEN""","""Z2309211401502…","""2023/201204""",0,2023-09-21,1,0,"""Rutiner og led…","""Rutiner og lei…",0,"""Lokaler og uts…","""Lokaler og uts…",1,"""Mat-håndtering…","""Mat-handtering…",1,"""Merking og spo…","""Merking og spo…",0,false,2023
"""Z1511031231512…",916144636,"""La Pergola""","""NEDRE LANGGATE…",null,"""3126""","""TØNSBERG""","""Z2309201455078…","""2023/189969""",0,2023-09-20,0,1,"""Rutiner og led…","""Rutiner og lei…",0,"""Lokaler og uts…","""Lokaler og uts…",0,"""Mat-håndtering…","""Mat-handtering…",0,"""Merking og spo…","""Merking og spo…",0,false,2023
"""Z2301241156064…",930215805,"""Bella Vista Ho…","""Prestegaten 2A…",null,"""3181""","""HORTEN""","""Z2309141558437…","""2023/195529""",0,2023-09-14,1,0,"""Rutiner og led…","""Rutiner og lei…",0,"""Lokaler og uts…","""Lokaler og uts…",1,"""Mat-håndtering…","""Mat-handtering…",0,"""Merking og spo…","""Merking og spo…",0,false,2023


In [26]:
#10  Finn tilsyn i Tønsberg eller Horten hvor det er gitt karakteren 2 eller 3 til Renhold. Her må du joine med kravpunktene. 
df_k = pl.read_csv("kravpunkter.csv", separator=";").with_columns(
    pl.col("dato").cast(str).str.pad_start(8, "0").str.to_date(format="%d%m%Y")
)
df_k

tilsynid,dato,ordningsverdi,kravpunktnavn_no,kravpunktnavn_nn,karakter,tekst_no,tekst_nn
str,date,f64,str,str,i64,str,str
"""Z1601041508412…",2016-01-04,1.1,"""Synlig rapport…","""Synleg rapport…",5,"""Ikke vurdert""","""Ikkje vurdert"""
"""Z1601041508412…",2016-01-04,1.2,"""Meldeplikt for…","""Meldeplikt for…",5,"""Ikke vurdert""","""Ikkje vurdert"""
"""Z1601041508412…",2016-01-04,1.3,"""Ansvaret til d…","""Ansvaret til d…",5,"""Ikke vurdert""","""Ikkje vurdert"""
"""Z1601041508412…",2016-01-04,1.4,"""Internkontroll…","""Internkontroll…",0,null,null
"""Z1601041508412…",2016-01-04,1.5,"""Farevurdering …","""Farevurdering …",5,"""Ikke vurdert""","""Ikkje vurdert"""
"""Z1601041508412…",2016-01-04,1.6,"""Opplæring og k…","""Opplæring og k…",5,"""Ikke vurdert""","""Ikkje vurdert"""
"""Z1601041508412…",2016-01-04,2.1,"""Generelt - pla…","""Generelt - pla…",2,"""Mangler ved hy…","""Manglar ved hy…"
"""Z1601041508412…",2016-01-04,2.2,"""Lokaler - spes…","""Lokaler - spes…",5,"""Ikke vurdert""","""Ikkje vurdert"""
"""Z1601041508412…",2016-01-04,2.3,"""Renhold""","""Reinhald""",0,null,null


In [28]:
df_t.filter(
    pl.col("poststed").is_in(["TØNSBERG", "HORTEN"])
).select("navn","orgnummer", "tilsynid").join(
    df_k.filter(
        pl.col("karakter").is_in([2,3])
    ).filter(
        pl.col("kravpunktnavn_no").eq(pl.lit("Renhold"))
    ).select("tilsynid", "karakter"),
    on = "tilsynid",
    how = "inner"
)

navn,orgnummer,tilsynid,karakter
str,i64,str,i64
"""La Pergola""",916144636,"""Z1601201546362…",2
"""Nøtterøy Baker…",974196506,"""Z1602031631190…",2
"""Pizzavera Solv…",984386931,"""Z1602120842515…",2
"""Fregatten sush…",971785551,"""Z1602251117043…",2
"""Fregatten sush…",971785551,"""Z1603010920011…",2
"""Rosalinda`s Or…",972876909,"""Z1604041455592…",2
"""Ms. Bubble Tea…",998183251,"""Z1605241651478…",3
"""Quality Hotel …",983138535,"""Z1609201130465…",2
"""Paparazzi""",994692038,"""Z1610201631027…",2


In [32]:
# 11 Finn tilsyn i Tønsberg eller Horten hvor det er gitt surmunn som totalkarakter, og samle opp kravpunktene som har fått surmunn 3 i en liste.
df_t.filter(
    pl.col("poststed").is_in(["TØNSBERG", "HORTEN"])
).filter(
    pl.col("surmunn")
).select(
    "navn", "orgnummer", "total_karakter", "tilsynid", "dato"
).join(
    df_k.filter(
        pl.col("karakter").eq(3)
    ).select("tilsynid", "karakter", "kravpunktnavn_no"),
    on = "tilsynid",
    how = "inner"
).group_by(
    "navn", "orgnummer", "tilsynid", "dato"
).agg(
    pl.col("kravpunktnavn_no")
)

navn,orgnummer,tilsynid,dato,kravpunktnavn_no
str,i64,str,date,list[str]
"""Ms. Bubble Tea…",998183251,"""Z1605241651478…",2016-05-24,"[""Generelt - planløsning, standard og vedlikehold"", ""Renhold"", … ""Tining""]"
"""Golden Crown""",972204668,"""Z1702021943311…",2017-02-02,"[""Generelt - planløsning, standard og vedlikehold"", ""Forurensningsfare"", ""Varmebehandling""]"
"""Fregatten sush…",971785551,"""Z1603010920011…",2016-02-29,"[""Generelt - planløsning, standard og vedlikehold"", ""Forurensningsfare""]"
"""NANA Thaikaway…",999372368,"""Z1706151853247…",2017-06-15,"[""Generelt - planløsning, standard og vedlikehold"", ""Renhold"", … ""Nedkjøling""]"
"""My Cafe""",989385690,"""Z2303011454051…",2023-03-01,"[""Råvarer"", ""Kjølekjeden""]"
"""Ms. Bubble Tea…",998183251,"""Z1712041709524…",2017-12-04,"[""Generelt - planløsning, standard og vedlikehold"", ""Renhold"", … ""Kjølekjeden""]"
"""Fregatten sush…",971785551,"""Z1602251117043…",2016-02-24,"[""Generelt - planløsning, standard og vedlikehold"", ""Forurensningsfare""]"
"""My Cafe""",989385690,"""Z2302231505002…",2023-02-23,"[""Råvarer"", ""Kjølekjeden""]"


In [38]:
# 12 
# I denne oppgaven skal du bruke lazy evaluering. 
# Start med å bruke scan_csv til å lese inn csvene. 
# Deretter skal du lage en rapport over seneste kontrollene i Tønsberg og Horten hvor det er gitt strekmunn eller surmunn. 
# Rapporten skal inneholde karakteren som er gitt, 
# en liste over kravpunktene som har strekmunn i en av kolonnene,
# en liste over kravpunktene som har surmunn i en av kolonnene. 
# Rapporten skal være sortert nyeste kontroll først.
fix_date = pl.col("dato").str.to_date(format="%d%m%Y")
lf_t = pl.scan_csv("tilsyn.csv", separator=";", dtypes={"postnr":str, "dato":str}).with_columns(
    fix_date
)
lf_k = pl.scan_csv("kravpunkter.csv", separator=";",  dtypes={"dato":str}).with_columns(
    fix_date
)
lf_t = lf_t.filter(
    pl.col("poststed").is_in(["TØNSBERG", "HORTEN"])
).filter(
    pl.col("total_karakter").is_in([2,3])
).select(
    "navn", "orgnummer", "total_karakter", "tilsynid", "dato"
)

lf_k = lf_k.select("tilsynid", "kravpunktnavn_no", "karakter")

lf_t = lf_t.join(lf_k, on="tilsynid")
lf_t = lf_t.group_by(
    "navn", "orgnummer", "total_karakter", "dato"
).agg(
    pl.col("kravpunktnavn_no").filter(pl.col("karakter").eq(3)).alias("surmunn_liste"),
    pl.col("kravpunktnavn_no").filter(pl.col("karakter").eq(2)).alias("strekmunn_liste")    
).sort(
    pl.col("dato"), descending=True
)
lf_t.collect()

navn,orgnummer,total_karakter,dato,surmunn_liste,strekmunn_liste
str,i64,i64,date,list[str],list[str]
"""Kontoret""",916942931,2,2024-01-11,[],"[""Internkontroll"", ""Generelt - planløsning, standard og vedlikehold"", … ""Kjølekjeden""]"
"""La Pergola""",916144636,2,2023-09-06,[],"[""Generelt - planløsning, standard og vedlikehold"", ""Bekjempe skadedyr""]"
"""Burger King - …",929545435,2,2023-07-20,[],"[""Forurensningsfare"", ""Kjølekjeden""]"
"""Roar i Bua""",979728468,2,2023-06-29,[],"[""Internkontroll"", ""Generelt - planløsning, standard og vedlikehold"", … ""Kjølekjeden""]"
"""Paparazzi""",994692038,2,2023-05-24,[],"[""Generelt - planløsning, standard og vedlikehold"", ""Renhold"", ""Kjølekjeden""]"
"""Jordbærpikene …",992699183,2,2023-04-21,[],"[""Generelt - planløsning, standard og vedlikehold"", ""Renhold""]"
"""ALMA""",922286108,2,2023-03-22,[],"[""Renhold"", ""Personaltoalett og garderobe""]"
"""URO på Haugar …",925813885,2,2023-03-09,[],"[""Internkontroll"", ""Personaltoalett og garderobe""]"
"""My Cafe""",989385690,3,2023-03-01,"[""Råvarer"", ""Kjølekjeden""]","[""Synlig rapport for smilefjes"", ""Internkontroll"", … ""Tining""]"


In [44]:
# 13. Utvid oppgaven over ved å inkludere totalkarakteren på forrige kontroll dersom den finnes i rapporten.
fix_date = pl.col("dato").str.to_date(format="%d%m%Y")
lf_t = pl.scan_csv("tilsyn.csv", separator=";", dtypes={"postnr":str, "dato":str}).with_columns(
    fix_date
).sort(
    ["orgnummer", "navn", "dato"], descending=[False, False, True],
).with_columns(
    pl.col("total_karakter").shift(-1).over(["orgnummer", "navn"]).alias("forrige_total_karakter")
)

lf_k = pl.scan_csv("kravpunkter.csv", separator=";",  dtypes={"dato":str}).with_columns(
    fix_date
)
lf_t = lf_t.filter(
    pl.col("poststed").is_in(["TØNSBERG", "HORTEN"])
).filter(
    pl.col("total_karakter").is_in([2,3])
).select(
    "navn", "orgnummer", "total_karakter", "tilsynid", "dato", "forrige_total_karakter"
)

lf_k = lf_k.select("tilsynid", "kravpunktnavn_no", "karakter")

lf_t = lf_t.join(lf_k, on="tilsynid")
lf_t = lf_t.group_by(
    "navn", "orgnummer", "total_karakter","forrige_total_karakter", "dato"
).agg(
    pl.col("kravpunktnavn_no").filter(pl.col("karakter").eq(3)).alias("surmunn_liste"),
    pl.col("kravpunktnavn_no").filter(pl.col("karakter").eq(2)).alias("strekmunn_liste")    
).sort(
    pl.col("dato"), descending=True
)
lf_t.collect()


navn,orgnummer,total_karakter,forrige_total_karakter,dato,surmunn_liste,strekmunn_liste
str,i64,i64,i64,date,list[str],list[str]
"""Kontoret""",916942931,2,0,2024-01-11,[],"[""Internkontroll"", ""Generelt - planløsning, standard og vedlikehold"", … ""Kjølekjeden""]"
"""La Pergola""",916144636,2,1,2023-09-06,[],"[""Generelt - planløsning, standard og vedlikehold"", ""Bekjempe skadedyr""]"
"""Burger King - …",929545435,2,null,2023-07-20,[],"[""Forurensningsfare"", ""Kjølekjeden""]"
"""Roar i Bua""",979728468,2,0,2023-06-29,[],"[""Internkontroll"", ""Generelt - planløsning, standard og vedlikehold"", … ""Kjølekjeden""]"
"""Paparazzi""",994692038,2,1,2023-05-24,[],"[""Generelt - planløsning, standard og vedlikehold"", ""Renhold"", ""Kjølekjeden""]"
"""Jordbærpikene …",992699183,2,0,2023-04-21,[],"[""Generelt - planløsning, standard og vedlikehold"", ""Renhold""]"
"""ALMA""",922286108,2,1,2023-03-22,[],"[""Renhold"", ""Personaltoalett og garderobe""]"
"""URO på Haugar …",925813885,2,0,2023-03-09,[],"[""Internkontroll"", ""Personaltoalett og garderobe""]"
"""My Cafe""",989385690,3,3,2023-03-01,"[""Råvarer"", ""Kjølekjeden""]","[""Synlig rapport for smilefjes"", ""Internkontroll"", … ""Tining""]"
